In [1]:
from ipyparallel import Client
import os
c = Client(profile='mpi')
v = c[:]
c.ids

[0, 1]

# Libraries

In [2]:
%%px
%matplotlib inline

import os
import time
import shutil
import numpy as np
import mpi4py.MPI as MPI

from matplotlib import cm
import matplotlib.pyplot as plt

import triangle

import pandas

from pyBadlands import FVmethod
from pyBadlands import diffLinear
from pyBadlands import raster2TIN 
from pyBadlands import flowNetwork
from pyBadlands import elevationTIN
from pyBadlands import partitionTIN
from pyBadlands import visualiseTIN
from pyBadlands import visualiseFlow

import warnings
warnings.filterwarnings("ignore")

# Initialise MPI communications
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Initialisation Phase

In [3]:
%%px
Init_time = time.clock()

In [4]:
%%px
# Digital elevation model file path
demfile = '/home/ubuntu/mpiPyLEM/data/regularLR.csv'
# Planchon & Darboux filling thickness limit [m]
fillmax = 1.
# Precipitation rate [m/a]
rainVal = 1.
# Sea level position [m]
seapos = -3450. #-1.e6
# Limit flow network computation based 
# on relative sea level position [m]
sealimit = 100.
# Diffusion coefficients [m2/a]
CDa = 0.5   # aerial
CDm = 0.5   # marine
# Stream power law coefficients
SPLm = 0.5
SPLn = 1.
# Stream power law erodibility [m^(1-2*SPLm)/a]
SPLero = 1.e-5 
# Minimum time step [a]
minDT = 1.
# Output folder path
outDir = '/home/ubuntu/mpiPyLEM/out'
# Time start [a]
tStart = 0.
# Time end [a]
tEnd = 100.
# Display interval [a]
tDisplay = 50.

## 1. Get DEM regular grid and create Badlands TIN.

In [5]:
%%px
walltime = time.clock()
tNow = tStart
recGrid = raster2TIN.raster2TIN(demfile,outDir,rank)

tinData = np.column_stack((recGrid.tinMesh['vertices'][:,0],
    recGrid.tinMesh['vertices'][:,1]))

if rank == 0:
    print " - read dataset and perform delaunay triangulation ", time.clock() - walltime

[stdout:0]  - read dataset and perform delaunay triangulation  0.351687


## 2. Partition the TIN

In [6]:
%%px
walltime = time.clock()

# Set parameters of the finite volume mesh
FVmesh = FVmethod.FVmethod(recGrid.tinMesh['vertices'],
                  recGrid.tinMesh['triangles'],
                  recGrid.tinMesh['edges'])

# Perform partitioning by equivalent domain splitting
partitionIDs, RowProc, ColProc = partitionTIN.simple(tinData[:,0], tinData[:,1])
FVmesh.partIDs = partitionIDs

# Get each partition global node ID
inGIDs = np.where(partitionIDs == rank)[0]

if rank == 0:
    print " - partition TIN amongst processors ", time.clock() - walltime

[stdout:0]  - partition TIN amongst processors  1.580268


## 3. Build Finite Volume discretisation

In [7]:
%%px

# Define overlapping partitions
allIDs, localTIN = partitionTIN.overlap(tinData[:,0], \
    tinData[:,1], RowProc, ColProc, 2*recGrid.resEdges)

# Set parameters of the finite volume mesh
tMesh = FVmethod.FVmethod(localTIN['vertices'],
                  localTIN['triangles'],
                  localTIN['edges'])

walltime = time.clock()

# Define Finite Volume parameters
totPts = len(tinData[:,0])
FVmesh.neighbours = np.zeros((totPts,20), dtype=np.int) 
FVmesh.neighbours.fill(-2)
FVmesh.edge_lenght = np.zeros((totPts,20), dtype=np.float) 
FVmesh.vor_edges = np.zeros((totPts,20), dtype=np.float) 
FVmesh.control_volumes = np.zeros(totPts, dtype=np.float) 

# Compute Finite Volume parameters
tGIDs, tNgbh, tEdgs, tVors, tVols = tMesh.construct_FV(inGIDs, allIDs, totPts)

FVmesh.neighbours[tGIDs,:tMesh.maxNgbh] = tNgbh
FVmesh.edge_lenght[tGIDs,:tMesh.maxNgbh] = tEdgs
FVmesh.vor_edges[tGIDs,:tMesh.maxNgbh] = tVors
FVmesh.control_volumes[tGIDs] = tVols

if rank == 0:
    print " - FV mesh ", time.clock() - walltime

[stdout:0] 
 - partition TIN including shadow zones  0.064733
 - build the voronoi diagram  0.082595
 - construct Finite Volume representation  0.241252
 - perform MPI communication  0.056408
 - FV mesh  0.480899


## 4. Interpolate elevation

In [8]:
%%px
walltime = time.clock()
inIDs = np.where(FVmesh.partIDs[recGrid.boundsPt:] == rank)[0]
inIDs += recGrid.boundsPt
 
local_elev = np.zeros(totPts)
local_elev.fill(-1.e6)
local_elev[inIDs] = elevationTIN.getElevation(recGrid.regX, recGrid.regY, \
    recGrid.regZ, FVmesh.node_coords[inIDs,:2])
comm.Allreduce(MPI.IN_PLACE, local_elev, op=MPI.MAX)

xyMin = [recGrid.regX.min(),recGrid.regY.min()]
xyMax = [recGrid.regX.max(),recGrid.regY.max()]

elevation = elevationTIN.update_border_elevation(local_elev, \
    FVmesh.neighbours, FVmesh.edge_lenght, recGrid.boundsPt, btype='slope')

# Define variables
cumdiff = np.zeros(totPts)
hillslope = diffLinear()
hillslope.CDaerial = CDa
hillslope.CDmarine = CDm 
flow = flowNetwork()
flow.erodibility = SPLero
flow.m = SPLm
flow.n = SPLn
flow.mindt = minDT
th5file = 'tin.time'
txmffile = 'tin.time'
txdmffile = 'tin.series.xdmf'
fh5file = 'flow.time'
fxmffile = 'flow.time'
fxdmffile = 'flow.series.xdmf'
    
if rank == 0:
    print " - interpolate elevation on grid ", time.clock() - walltime

[stdout:0]  - interpolate elevation on grid  0.045608


In [9]:
%%px
if rank == 0:
    print " - Initialisation phase ", time.clock() - Init_time

[stdout:0]  - Initialisation phase  2.6531


# Main loop

In [10]:
%%px
sealevel = seapos
rain = np.zeros(totPts, dtype=float)
rain.fill(rainVal)
rain[:recGrid.boundsPt] = 0.
tDisp =  tStart + tDisplay
step = 0
extTime = tDisp

# Flow computation

In [11]:
%%px
tNow = tStart
Flow_time = time.clock()

## 1. Perform pit filling

In [12]:
%%px
walltime = time.clock()
fillH = elevationTIN.pit_filling_PD(elevation, FVmesh.neighbours, \
                recGrid.boundsPt, sealevel-sealimit, fillmax ,0.01)
if rank == 0:
    print " - depression-less algorithm PD with stack", time.clock() - walltime

[stdout:0]  - depression-less algorithm PD with stack 0.036225


## 2. Compute stream network

In [13]:
%%px
walltime = time.clock()
ngbhs = FVmesh.neighbours[allIDs,:]
edges = FVmesh.vor_edges[allIDs,:]
distances = FVmesh.edge_lenght[allIDs,:]
flow.SFD_receivers(fillH, elevation, ngbhs, edges, distances, allIDs, sealevel-sealimit)
if rank == 0:
    print " - compute receivers parallel ", time.clock() - walltime

# Distribute evenly local minimas to processors
walltime = time.clock()
flow.localbase = np.array_split(flow.base, size)[rank]
flow.ordered_node_array()
if rank == 0:
    print " - compute stack order locally ", time.clock() - walltime
    
walltime = time.clock()
stackNbs = comm.allgather(len(flow.localstack))
globalstack = np.zeros(sum(stackNbs),dtype=flow.localstack.dtype)
comm.Allgatherv(sendbuf=[flow.localstack, MPI.INT], 
             recvbuf=[globalstack, (stackNbs, None), MPI.INT]) 
flow.stack = globalstack
if rank == 0:
    print " - send stack order globally ", time.clock() - walltime

[stdout:0] 
 - compute receivers parallel  0.111561
 - compute stack order locally  0.012323
 - send stack order globally  0.001522


## 3. Compute discharge

In [14]:
%%px
walltime = time.clock()
flow.compute_flow(FVmesh.control_volumes, rain, True)
if rank == 0:
    print " - compute discharge ", time.clock() - walltime

[stdout:0]  - compute discharge  0.013862


## 4. Compute CFL condition

In [15]:
%%px
walltime = time.clock()  
hillslope.dt_pstability(FVmesh.edge_lenght[inGIDs,:tMesh.maxNgbh])
#
flow.dt_fstability(FVmesh.node_coords[:,:2],fillH,inGIDs)
#
CFLtime = min(flow.CFL,hillslope.CFL)
CFLtime = max(minDT,CFLtime)
if rank == 0:
    print " - Get CFL time step ", time.clock() - walltime

[stdout:0]  - Get CFL time step  0.015952


## 5. Compute sediment fluxes

In [16]:
%%px
# Initial cumulative elevation change
walltime = time.clock()

diff_flux = hillslope.sedflux(flow.diff_flux,sealevel,elevation,FVmesh.control_volumes)
if rank == 0:
    print " - Get hillslope fluxes ", time.clock() - walltime

walltime = time.clock()
tstep,sedrate = flow.compute_sedflux(FVmesh.control_volumes,elevation,fillH,
                FVmesh.node_coords[:,:2],xyMin,xyMax,diff_flux,CFLtime,sealevel,True)
if rank == 0:
    print " - Get stream fluxes ", time.clock() - walltime

# Update surface
timestep = min(tstep,extTime-tNow)
elevation = elevation + sedrate * timestep
cumdiff = cumdiff + sedrate * timestep

[stdout:0] 
 - Get hillslope fluxes  0.003065
 - Get stream fluxes  0.005721


In [17]:
%%px
if rank == 0:
    print " - Flow computation ", time.clock() - Flow_time

[stdout:0]  - Flow computation  0.266899


# Create Hdf5 outputs

we first define local vertices, triangles and polylines used to create the visualisation outputs.

In [18]:
%%px
out_time = time.clock()
visXlim = np.zeros(2)
visYlim = np.zeros(2)
visXlim[0] = recGrid.rectX.min()
visXlim[1] = recGrid.rectX.max()
visYlim[0] = recGrid.rectY.min()
visYlim[1] = recGrid.rectY.max()

# Done when TIN has been built/rebuilt
outPts, outCells = visualiseTIN.output_cellsIDs(allIDs,inIDs,visXlim,visYlim,
                                        FVmesh.node_coords[:,:2],tMesh.cells)
tcells = np.zeros(size)
tcells[rank] = len(outCells)
comm.Allreduce(MPI.IN_PLACE,tcells,op=MPI.MAX)
tnodes = np.zeros(size)
tnodes[rank] = len(allIDs)
comm.Allreduce(MPI.IN_PLACE,tnodes,op=MPI.MAX)

# Done for every visualisation step
flowIDs, polylines = visualiseFlow.output_Polylines(outPts,flow.receivers[outPts],
                visXlim,visYlim,FVmesh.node_coords[:,:2])
fnodes = np.zeros(size)
fnodes[rank] = len(flowIDs)
comm.Allreduce(MPI.IN_PLACE,fnodes,op=MPI.MAX)
fline = np.zeros(size)
fline[rank] = len(polylines[:,0])
comm.Allreduce(MPI.IN_PLACE,fline,op=MPI.MAX)

we then write to each partition simulation outputs (TIN & flow network) using HdF5.

In [19]:
%%px
th5file = 'tin.time'
txmffile = 'tin.time'
txdmffile = 'tin.series.xdmf'

fh5file = 'flow.time'
fxmffile = 'flow.time'
fxdmffile = 'flow.series.xdmf'

# Compute flow parameters
flow.compute_parameters(FVmesh.node_coords[:,:2])

# Write HDF5 files
visualiseTIN.write_hdf5(outDir,th5file,step,tMesh.node_coords[:,:2],elevation[allIDs],
                        flow.discharge[allIDs],cumdiff[allIDs],outCells,rank)
visualiseFlow.write_hdf5(outDir,fh5file,step,FVmesh.node_coords[flowIDs,:2],elevation[flowIDs],
                        flow.discharge[flowIDs],flow.chi[flowIDs],flow.basinID[flowIDs],polylines,rank)

# Combine HDF5 files and write time series
if rank==0:
    visualiseTIN.write_xmf(outDir,txmffile,txdmffile,step,tStart,tcells,tnodes,th5file,size)
    visualiseFlow.write_xmf(outDir,fxmffile,fxdmffile,step,tStart,fline,fnodes,fh5file,size)
    print " - Writing outputs ", time.clock() - out_time

[stdout:0]  - Writing outputs  0.410806


In [ ]:
%%px
print flowIDs